In [2]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

c:\Users\Owner\miniconda3\envs\stats\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\Owner\miniconda3\envs\stats\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
query = """
WITH prod as (
  SELECT Event_Date,User_ID, JSON_EXTRACT_ARRAY(Semantic_Event_Properties, '$.products') AS products
  FROM `ballosodeuk.airbridge_lake.app`
  WHERE  
    TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) >= TIMESTAMP("2024-04-29") AND
    Event_Category = 'Ad Impression (App)' AND Event_Label = '홈_상단_캐시버튼' AND Event_Action = '직광고_쿠팡' And
    Semantic_Event_Properties is not Null AND
    Event_Date between "2024-04-11" AND "2024-04-24"
    ) 
    ,

click as (
  select distinct User_ID
  from `ballosodeuk.airbridge_lake.app`
  where 
    TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) >= TIMESTAMP("2024-04-29") AND 
    Event_Category = 'Ad Click (App)' AND
    Event_Date between "2024-04-11" AND "2024-04-24"
)
  
select p.Event_Date, p.User_ID,
  ARRAY_TO_STRING(ARRAY(
  select JSON_EXTRACT_SCALAR(products, '$.name')
  from UNNEST(p.products) As products
  ),',') as product_names,
  IF(p.Event_Date between "2024-04-11" AND "2024-04-17",'before','after') as period
from prod p
join 
  click c on p.User_ID = c.User_ID

"""

In [5]:
job_config = bigquery.QueryJobConfig()
job_config.use_query_cache = False
job_config.allow_large_results = True

In [6]:
query_job = bq.query(query,job_config=job_config)

In [ ]:
# 쿼리 실행 및 결과 저장
query_job = bq.query(query)

# 결과를 날짜별로 Parquet 파일로 저장
try:
    for partition in query_job.result().to_dataframe_iterable():
        if not partition.empty:
            # 이벤트 날짜를 YYYY-mm-dd 형식으로 변환하여 파일명 생성
            partition_date = partition['Event_Date'].iloc[0]
            # 저장할 경로 확인 및 생성
            save_path = f'../result/{partition_date}.parquet'
            # os.makedirs(os.path.dirname(save_path), exist_ok=True)
            # Parquet 파일로 저장
            partition.to_parquet(save_path)
except Exception as e:
    print(f'error: {e}')